2a

Dataset and DataLoader

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import numpy as np
import time
import requests
from torchvision.datasets import ImageFolder
from torchvision import models
from PIL import Image
from shutil import copyfile
import os

In [ ]:
os.chdir('/content/drive/MyDrive/MLDL_project')
from datasets.cityscapes import CityscapesDataset
from datasets.gta5 import GTA5

In [ ]:
import torchvision.transforms as transforms
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
train_dataset = CityscapesDataset('/content/drive/MyDrive/MLDL_project/datasets/Cityspaces', transform=transform, split='train')
val_dataset = CityscapesDataset('/content/drive/MyDrive/MLDL_project/datasets/Cityspaces', transform=transform, split='val')

In [ ]:
# Create a DataLoader
from torch.utils.data import DataLoader
dataloader_train = DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=8)
dataloader_val = DataLoader(val_dataset, batch_size=2, shuffle=False, num_workers=8)

In [ ]:
num_train_samples = len(train_dataset)
num_val_samples = len(val_dataset)

print(f'Number of training samples: {num_train_samples}')
print(f'Number of validation samples: {num_val_samples}')

Number of training samples: 1572
Number of validation samples: 500


Build model

In [ ]:
# Load pre-trained model

from models.deeplabv2.deeplabv2 import get_deeplab_v2
model = get_deeplab_v2().cuda()

Deeplab pretraining loading...


Trainning and validation process

In [ ]:
# This is just used to test if you can train your model with this dataloader
# from utils import poly_lr_scheduler
# from utils import fast_hist
# from utils import per_class_iou
# from utils import total_hist

# def total_hist(outputs, labels, num_classes):
#     hist = 0
#     for i in range(len(outputs)):
#         output, label = outputs[i].cpu().detach().numpy().reshape(-1,), labels[i].cpu().detach().numpy().reshape(-1,)
#         hist += fast_hist(label, output, num_classes)
#     return hist
#
# model.train()
# running_loss = 0.0
# hist = 0
# criterion = nn.CrossEntropyLoss(ignore_index=255)
# optimizer = optim.Adam(model.parameters(), lr=0.0001)  #Since our batch size is only 2, so we need to choose a small initial learning rate

# for i, (inputs, labels) in enumerate(dataloader_train, 0):
#     inputs, labels = inputs.cuda(), labels.cuda()
#     optimizer.zero_grad()

#     outputs, _, _ = model(inputs)
#     print(outputs.shape, labels.shape)
#     loss = criterion(outputs, labels)
#     loss.backward()
#     optimizer.step()

#     running_loss += loss.item()
#     outputs = torch.argmax(outputs, dim=1)
#     hist += total_hist(outputs, labels, 19)
#     break
# avg_loss = running_loss
# miou = np.mean(per_class_iou(hist))
# print(f"Avg. Training Loss: {avg_loss}, mIoU: {miou}")

torch.Size([2, 19, 512, 1024]) torch.Size([2, 512, 1024])


/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Avg. Training Loss: 6.983089447021484, mIoU: 0.012453069959973593


In [ ]:
from train import train
from validate import validation

In [ ]:
def poly_lr_scheduler(optimizer, init_lr=0.0001, init_lr_cls=0.001, iter=0, lr_decay_iter=1,
                      max_iter=300, power=0.9):
    """Polynomial decay of learning rate
            :param init_lr is base learning rate
            :param init_lr_cls is base learning rate for final classifier layer
            :param iter is a current iteration
            :param lr_decay_iter how frequently decay occurs, default is 1
            :param max_iter is number of maximum iterations
            :param power is a polymomial power

    """
    # if iter % lr_decay_iter or iter > max_iter:
    # 	return optimizer

    lr = init_lr*(1 - iter/max_iter)**power
    optimizer.param_groups[0]['lr'] = lr

    lr_cls = init_lr_cls*(1 - iter/max_iter)**power
    optimizer.param_groups[1]['lr'] = lr_cls
    return lr, lr_cls

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss(ignore_index=255)
optimizer = optim.SGD([
    {'params': [param for name, param in model.named_parameters() if 'layer6' not in name]},
    {'params': model.layer6.parameters(), 'lr': 0.001}
], lr=0.0001, momentum=0.9, weight_decay=0.0005)

# training model
epochs = 50

miou_train_list = []
miou_val_list = []
models = []
for epoch in range(epochs):
    print('--------------------------------------------------------------------------------')
    _, miou_train, miou_per_class_train = train(epoch, model, dataloader_train, criterion, optimizer)
    lr, lr_cls = poly_lr_scheduler(optimizer = optimizer, init_lr=0.0001, init_lr_cls=0.001, iter=epoch, lr_decay_iter=1, max_iter=epochs, power=0.9)
    # every 5 epochs print the miou and loss of validation set
    if epoch % 5 == 0:
      _, miou_val, miou_per_class_val = validation(model, dataloader_val, criterion)
      print('--------------------------------------------------------------------------------')
      miou_train_list.append(miou_train)
      miou_val_list.append(miou_val)
      models.append(model.state_dict())


print('Finished Training')

--------------------------------------------------------------------------------


/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch1 Avg. Training Loss: 0.4060953312171931, mIoU: 0.4208803753802576
Avg. Validation Loss: 0.2923396674692631, mIoU: 0.49425985346824197
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------


KeyboardInterrupt: 

In [ ]:
# save best model
import os
DIR = '/content/drive/MyDrive/MLDL_project/models/deeplabv2/trained_models/'
if not os.path.exists(DIR):
    os.makedirs(DIR)
PATH = DIR + f'deepLabV2_epoch{epochs}.pth'

# delete old model files
if os.path.exists(PATH):
    os.remove(PATH)

model = get_deeplab_v2().cuda()
model.load_state_dict(models[np.argmax(np.array(miou_val_list))])
torch.save(model.state_dict(), PATH)

In [ ]:
# visualization
import matplotlib.pyplot as plt

epochs_list = np.arange(1, epochs+1, 5)
plt.figure(figsize=(10, 6))

plt.plot(epochs_list, miou_train_list, marker='o', linestyle='-', color='r', label='Training mIOU')
plt.plot(epochs_list, miou_val_list, marker='o', linestyle='--', color='b', label='Val mIOU')

plt.title('Training and validation mIOU over Epochs')
plt.xlabel('Epochs')
plt.ylabel('mIOU')

plt.legend()
plt.grid(True)
plt.show()

Flops and Number of parameters

In [ ]:
!pip install -U fvcore

In [ ]:
from fvcore.nn import FlopCountAnalysis, flop_count_table

height = train_dataset[0][0].shape[0]
width = train_dataset[0][0].shape[1]
image = torch.zeros((1, 3, height, width)).cuda()

flops = FlopCountAnalysis(model, image)
print(flops)

total_params = sum(p.numel() for p in model.parameters())
print(f'Total number of parameters: {total_params}')

In [ ]:
#more detail information about number of parameters and flops
print(flop_count_table(flops))

Latency and FPS

In [ ]:
# latency and FPS
import time

height = train_dataset[0][0].shape[0]
width = train_dataset[0][0].shape[1]
image = np.random.randint(0,256,(height, width, 3)) / 255.
image = transform(image)
image = torch.unsqueeze(image, dim=0).float().cuda()

iterations = 1000
latency = np.zeros(iterations)
fps = np.zeros(iterations)
for i in range(iterations):
  start = time.time()
  output = model(image)
  end = time.time()
  time_diff_seconds = end - start
  latency[i] = time_diff_seconds
  fps[i] = 1/time_diff_seconds

meanLatency = np.mean(latency)*1000
stdLatency = np.std(latency)*1000
meanFPS = np.mean(fps)
stdFPS = np.std(fps)

print(f"Mean Latency: {meanLatency} ms")
print(f"Std Latency: {stdLatency} ms")
print(f"Mean FPS: {meanFPS}")
print(f"Std FPS: {stdFPS}")